# SQL 4 Data Science: Predict Project
## Bhejane Online Trading Store

### Student Starter Notebook

© Explore Data Science Academy

### Honour Code

I {**STELLA**, **NJUKI**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem. 

Its time to flex your Ninja SQL skills! 
 
**Your mission, should you choose to accept it:**

You will receive a denormalised database consisting of two tables. Your task is to transform the database such that is in third normal form (3NF). To ensure that a consistent normalisation process is followed by each student, we have attached an Entity Relationship Diagram (ERD) which is a guideline on what tables need to be produced. Once you have normalised the database, you will be required to complete an MCQ test found under the 'Predict' tab of Athena.  

Though the normalisation process, you will be guided into acknowledging client (Bhenjane Trading) use-cases for the database, and dealing with data anomalies in SQL. 

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
Please use the below command to install sql_magic, this is the package that will assist you with SQL syntax hightlighting.
* pip install sql_magic

Remember to start each new cell with " %%read_sql "

In [1]:
pip install sql_magic

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pickleshare in c:\users\stella\anaconda\lib\site-packages (from ipython->sql_magic) (0.7.5)



In [1]:
# DO NOT EDIT THIS CELL
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [2]:
# DO NOT EDIT THIS CELL
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [3]:
#DO NOT EDIT THIS CELL
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [4]:
%%read_sql
--DO NOT EDIT THIS CELL
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 11:11:03 PM E. Africa Standard Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

Although we have not yet covered python in the sprint at EXPLORE - you have additional resources at your disposal, one of which happens to be a data engineering expert, and she has created a brief script for you which can be used to extract the data from the csv files, and load it into a sqlite database. 

This will be included in your submission, so that the end-to-end processing of the data is visible, and repeatable for any additional consultants who may be brought on board at a later stage. 


In [5]:
# DO NOT EDIT CELL
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Explore the Denormalized Tables
Familiarize yourself with the data given in the two tables by writing your own SQL queries to explore properties of the dataset.i.e Look for data inconsistencies, anormalies, redundancies etc to guide your normalization process. 

This section will not be graded - think of it as your own scratch pad.



In [6]:
%%read_sql
-- #Use this section to explore the database by writing your own queries
SELECT * FROM Products
LIMIT 50;

Query started at 11:11:08 PM E. Africa Standard Time; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,1 x Hikvision 1080P Bullet camera,399.0
1,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,Manual,399.0
2,,,,10325354918,467.0,ZEE,Fashion / Accessories / Scarves,Grey,,ZEE 3-in-1 Unisex Gaiter,,0.0,Limited (6 months),,,139.0
3,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Makeup / Face / Found...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
4,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Shop By Brand / Estee...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
5,,,,27131187035,275.0,Estee Lauder,Beauty / Makeup / Face / Foundation,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
6,,,,39800085139,247.0,Energizer,Cameras / Cameras & Lenses / Camera Accessorie...,,South Africa,Energizer 3V CR2450 Lithium Coin Battery,,0.0,Limited (12 months),,,49.0
7,,,,43859499182,386.0,Fellowes,Office & Stationery / Stationery / Office Supp...,,South Africa,Fellowes HFC Free Air Duster - 350ml,,0.0,Limited (6 months),,1 x 350ml Free Air Duster,149.0
8,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0
9,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0


In [7]:
%%read_sql
SELECT * FROM transactions
LIMIT 5

Query started at 11:11:09 PM E. Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,Total,UserName,InvoiceDate
0,102,300507946,1523.0,DIMPHO,2020-07-02 0:00:00
1,1,43859499182,149.0,Hendrik,2020-08-05 0:00:00
2,2,614143543746,99.0,Faristha,2020-07-29 0:00:00
3,179,617566827837,3464.0,Zanele,2020-04-04 0:00:00
4,136,619659097318,3301.0,Junaid,2020-08-04 0:00:00


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below below target ERD create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


It is suggested that you create all your tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form
Use the above ERD sketch to create the required tables. Please label your tables as they appear in the ERD sketch

In [8]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 11:11:12 PM E. Africa Standard Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [9]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height 
FROM 
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;
    

Query started at 11:11:14 PM E. Africa Standard Time; Query executed in 0.00 m

###  7. Convert the database into its 2nd Normal Form (2NF).

Please label your tables as they appear in the ERD sketch.

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example, 

Thereby, we remove the partial dependancy, and will be in 2NF. We are instructed to create a unique identifier for the `Transactions_2NF`. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach. 

#### 7.3 Data Anomalies

You may stumble into an error when you insert the data which violates this constraint. You query this with the data owner, and you are told that there are three transactions in the database which were errors. You are informed that *all transactions in the database correspond to products are in the database*.

These entries were captured incorrectly, and must be removed. How will you deal with this? 

In [10]:
%%read_sql
select distinct(barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 11:11:16 PM E. Africa Standard Time; Query executed in 0.00 m

,Barcode
0,889899982693
1,6007226069631


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [11]:
%%read_sql
SELECT barcode 
FROM Transactions_1NF
WHERE barcode = 889899982693 OR barcode = 6007226069631

Query started at 11:11:18 PM E. Africa Standard Time; Query executed in 0.00 m

,Barcode
0,889899982693
1,6007226069631
2,6007226069631


In [12]:
%%read_sql
delete from Transactions_1NF where barcode = 889899982693 OR barcode = 6007226069631

Query started at 11:11:18 PM E. Africa Standard Time; Query executed in 0.00 m

In [13]:
%%read_sql
-- Should show no entries!
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 11:11:19 PM E. Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that you understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint yourself!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 

You will need to work around this by removing the erroneous data from the tables before continuing, since this would violate the FK constraint between the Transactions_2NF and Products_2NF table otherwise. 

_Hint: Identify the observations in the Transactions_1NF table, and remove them from there._

#### 7.5 Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database. 

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. You are tasked with removing 'useless' entries as you take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

_Hint: Make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [14]:
%%read_sql 

SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty
FROM
    Products
WHERE ItemDescription = '';

Query started at 11:11:21 PM E. Africa Standard Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


In [15]:
%%read_sql
SELECT colour 
FROM products
WHERE colour = '';

Query started at 11:11:22 PM E. Africa Standard Time; Query executed in 0.00 m

,Colour
0,
1,
2,
3,
4,
...,...
659,
660,
661,
662,


### Let's do it! Construct the database above in second normal form

In [16]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 11:11:24 PM E. Africa Standard Time; Query executed in 0.00 m

### Let's do it! _Populate_ the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries. 

In [17]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Products_1NF;
    
INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;
    
INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT 
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 11:11:27 PM E. Africa Standard Time; Query executed in 0.00 m

A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [18]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg, 
    Volume_litre, 
    Length,
    Width,
    Height 
FROM 
    Products_1NF
LEFT JOIN Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
LEFT JOIN PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 11:11:28 PM E. Africa Standard Time; Query executed in 0.01 m

Last but not least, we insert into the `Transactions_2NF`table. Recall that we created a new PK in this table, and pay attention to how we are inserting values into the table. 

In [19]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 11:11:30 PM E. Africa Standard Time; Query executed in 0.00 m

<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

Please label your tables as they appear in the ERD sketch. In the same fashion as done in 2NF, note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [20]:
%%read_sql

--#Create tables required for 2NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    "UserName" VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);

CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

Query started at 11:11:35 PM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the construction of the remaining tables mentioned </font>

In [21]:
%%read_sql

CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserID" VARCHAR(150) NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName"),
    CONSTRAINT fk_trans FOREIGN KEY('UserID') REFERENCES Users_3NF ('UserID'),
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_3NF ('Barcode'),
    CONSTRAINT fk_trans FOREIGN KEY('CartID') REFERENCES Carts_3NF ('CartID')
);

CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);

CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR (150),
    PRIMARY KEY ("BrandID" AUTOINCREMENT)
);

CREATE TABLE "Locations_3NF" (
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR (150),
    PRIMARY KEY ("LocationID" AUTOINCREMENT)
);

Query started at 11:11:37 PM E. Africa Standard Time; Query executed in 0.00 m

In [23]:
%%read_sql

--# Populate the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName FROM Transactions_2NF
    ;
    
INSERT INTO "Colours_3NF" ("Colour")
SELECT DISTINCT 
    Colour
FROM Colours_2NF
    ;
    
INSERT INTO "Carts_3NF" ("CartID","InvoiceDate","Total")
SELECT DISTINCT 
    CartID,
    InvoiceDate,
    Total
FROM Transactions_2NF
    ;
    
INSERT INTO "Brands_3NF" ("Brand")
SELECT DISTINCT 
    Brand
FROM Products_2NF
WHERE Brand <> ''
    ;
    
INSERT INTO "Locations_3NF" ("StockCountry")
SELECT DISTINCT 
    StockCountry
FROM Products_2NF
WHERE StockCountry <> ''
    ;

INSERT INTO "Navigation_3NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Navigation_2NF
    ;
    
INSERT INTO "PackageContents_3NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM 
    PackageContents_2NF
    ;


INSERT INTO "Products_3NF" ("Barcode","ProductDescription","Price","Quantity","Weight_kg","Volume_litre","Length",
                            "Width","Height","PathID","ItemID","ColourID","BrandID","LocationID")
SELECT 
    Barcode,
    ProductDescription,
    Price,
    Quantity,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height,
    PathID,
    ItemID,
    ColourID,
    BrandID,
    LocationID
FROM Products_2NF
LEFT JOIN Brands_3NF ON Products_2NF.Brand = Brands_3NF.Brand
LEFT JOIN Locations_3NF ON Products_2NF.StockCountry = Locations_3NF.StockCountry
    ;
    
INSERT INTO "Transactions_3NF" ("CartID||Barcode||UserName","CartID","Barcode","UserID")
SELECT DISTINCT 
    CartID||Transactions_2NF.Barcode||Transactions_2NF.UserName,
    CartID,
    Barcode,
    UserID
FROM
    Transactions_2NF
LEFT JOIN Users_3NF ON Transactions_2NF.UserName = Users_3NF.UserName
    ;


Query started at 11:11:51 PM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Carts_3NF`.   </font>

Take note that there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

### <font color='turquoise'>Action: proceed with the insertion into the tables below. </font>

Similarly to the table above, mind the occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows. 


### <font color='turquoise'>Action: proceed with the insertion into `Products_3NF`. </font>
Take note of the joins required for additional data contained in other tables. 

### <font color='turquoise'>Action: proceed with the insertion into `Transactions_3NF`.   </font>

Follow previous proceedure, and checks. 

### <font color='turquoise'>Action: Investigate the DB.   </font>

Note - the tables counts below are what you should arrive at. If you don't, reconsider how you have done the inserts above. Having the tables correctly formatted will allow to answer the questions more easily, and without errors.

| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [25]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
select 'Products_3NF' as table_name, count(*) from Products_3NF
UNION
select 'Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select 'Users_3NF' as table_name, count(*) from Users_3NF
UNION
select 'Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select 'PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select 'Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select 'Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select 'Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select 'Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 11:12:07 PM E. Africa Standard Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,396
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


<a id='MCQ_questions'></a>
## 9. MCQ Questions

Having completed the normalisation of the database, you may use the following cells to help you answer the MCQs for the predict. 

Q1) How many unique products does the company have?

In [26]:
%%read_sql
SELECT COUNT (DISTINCT barcode)
FROM products_3NF;


Query started at 11:12:14 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT (DISTINCT barcode)
0,608


Q2) How many users bought from Bhejane in April 2020?

In [27]:
%%read_sql
SELECT COUNT (DISTINCT UserID) 
FROM transactions_3NF
LEFT JOIN carts_3NF
ON transactions_3NF.CartID = carts_3NF.CartID
WHERE carts_3NF.InvoiceDate LIKE "2020-04%";

Query started at 11:12:35 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT (DISTINCT UserID)
0,31


Q3) How many users bought 3 or more items that cost more than R1000?

In [56]:
%%read_sql
SELECT  T_3NF.CartID, COUNT(T_3NF.CartID) AS count_cartid, T_3NF.Barcode,T_3NF.UserID,C_3NF.InvoiceDate, C_3NF.total
FROM Transactions_3NF AS T_3NF
LEFT JOIN Carts_3NF AS C_3NF
ON T_3NF.CartID = C_3NF.CartID
WHERE total > 1000
GROUP BY T_3NF.CartID
ORDER BY count_cartid DESC
LIMIT 50;

Query started at 11:43:06 PM E. Africa Standard Time; Query executed in 0.00 m

,CartID,count_cartid,Barcode,UserID,InvoiceDate,Total
0,172,4,6009900463131,142,2020-07-24 0:00:00,1114.0
1,160,4,4015400541813,134,2020-04-17 0:00:00,1134.0
2,136,4,619659097318,112,2020-08-04 0:00:00,3301.0
3,115,4,6009684040641,97,2020-06-13 0:00:00,1716.0
4,102,4,300507946,87,2020-07-02 0:00:00,1523.0
5,179,3,617566827837,6,2020-04-04 0:00:00,3464.0
6,156,3,718037858494,64,2020-06-07 0:00:00,1351.0
7,139,3,6009706869533,115,2020-06-27 0:00:00,1177.0
8,114,3,6002322014646,96,2020-05-31 0:00:00,1137.0
9,113,3,6003000640348,95,2020-08-11 0:00:00,2535.0


Q4) Which user made the largest purchase on a single transaction?

In [59]:
%%read_sql
SELECT T_3NF.CartID, T_3NF.Barcode, T_3NF.UserID, C_3NF.InvoiceDate, C_3NF.Total, U_3NF.username
FROM Transactions_3NF AS T_3NF
LEFT JOIN Carts_3NF AS C_3NF
ON T_3NF.CartID = C_3NF.CartID
LEFT JOIN Users_3NF AS U_3NF
ON T_3NF.UserID = U_3NF.UserID
ORDER BY total DESC
LIMIT 5

Query started at 11:52:29 PM E. Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,UserID,InvoiceDate,Total,UserName
0,123,HTB5ABG31E3,64,2020-04-11 0:00:00,10999.0,Mandla
1,124,HTB5ABG31E3,83,2020-05-30 0:00:00,10999.0,Brigette
2,125,HTB5FI3BC8C,24,2020-03-27 0:00:00,3765.0,Andreas Peter
3,126,HTB5FI3BC8C,104,2020-08-12 0:00:00,3765.0,Mzoxolo
4,179,617566827837,6,2020-04-04 0:00:00,3464.0,Zanele


Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [66]:
%%read_sql
SELECT *
FROM packagecontents_3NF
LIMIT 5;

Query started at 11:58:49 PM E. Africa Standard Time; Query executed in 0.00 m

,ItemID,ItemDescription,PackType,Warranty
0,1,1 x Hikvision 1080P Bullet camera,,Limited (6 months)
1,2,Manual,,Limited (6 months)
2,3,1 x 350ml Free Air Duster,,Limited (6 months)
3,4,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,Limited (6 months)
4,5,Wahl Easy Cut 15 Piece Complete Hair Clipper Kit,,Limited (6 months)


In [68]:
%%read_sql
SELECT P_3NF.barcode, P_3NF.productdescription, P_3NF.price, PC_3NF.ItemDescription, P_3NF.ItemID
FROM Products_3NF AS P_3NF
LEFT JOIN Packagecontents_3NF AS PC_3NF
ON P_3NF.ItemID = PC_3NF.ItemID
WHERE productdescription LIKE "%5m Colour Changing RGB LED Strip Light%" AND barcode = 'MPTAL57588104'
LIMIT 50


Query started at 12:06:28 AM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,ProductDescription,Price,ItemDescription,ItemID
0,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 5 Meter LED RGB Strip Light,516
1,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 24 key IR remote controller,517
2,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x IR Controller,518
3,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 12V 3A Power Supply,519
4,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 5 Meter LED RGB Strip Light,516
5,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 24 key IR remote controller,517
6,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x IR Controller,518
7,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,1 x 12V 3A Power Supply,519


Q6) How many brands are available at Bhejane?

In [71]:
%%read_sql
SELECT COUNT(DISTINCT Brand)
FROM brands_3NF
LIMIT 50;


Query started at 12:10:07 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT Brand)
0,232


7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [74]:
%%read_sql
SELECT *
FROM  Products_3NF
WHERE ProductDescription LIKE "%Verimark - Floorwiz 2in1 Mop%"
LIMIT 50;


Query started at 12:12:08 AM E. Africa Standard Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,537,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,236,None,None,1
1,538,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,237,None,None,1
2,539,6005427110503,Verimark - Floorwiz 2in1 Mop,179.0,104,,0.0,90.0,14.0,2.0,182,238,None,None,1


Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [77]:
%%read_sql
SELECT Barcode,ProductDescription,Price, Quantity, Length, Width, Height, Length*Width*Height AS Vol
FROM  Products_3NF
WHERE ProductDescription LIKE "%Russell Hobbs - Slow Cooker%"
LIMIT 5;


Query started at 12:17:07 AM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,ProductDescription,Price,Quantity,Length,Width,Height,Vol
0,6002322013823,Russell Hobbs - Slow Cooker,599.0,270,24.0,34.0,"33,4",26928.0
1,6002322013823,Russell Hobbs - Slow Cooker,599.0,270,24.0,34.0,"33,4",26928.0
2,6002322013823,Russell Hobbs - Slow Cooker,599.0,270,24.0,34.0,"33,4",26928.0
3,6002322013823,Russell Hobbs - Slow Cooker,599.0,270,24.0,34.0,"33,4",26928.0


Q9) Which user made the most transactions in the Year 2020?

In [91]:
%%read_sql
SELECT * , COUNT (UserID) AS counted
FROM Transactions_3NF AS T_3NF
LEFT JOIN carts_3NF AS C_3NF
ON T_3NF.cartID = C_3NF.cartID
WHERE InvoiceDate LIKE "2020%"
GROUP BY userID
ORDER BY counted DESC
LIMIT 50


Query started at 12:27:55 AM E. Africa Standard Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserID,CartID,InvoiceDate,Total,counted
0,114015400541813Cornelis,11,4015400541813,9,11,2020-05-19 0:00:00,459.0,9
1,366001651058932Erlo,36,6001651058932,31,36,2020-08-10 0:00:00,99.0,8
2,6718037855448Zanele,6,718037855448,6,6,2020-06-01 0:00:00,840.0,6
3,1446003977000206Heather,144,6003977000206,120,144,2020-04-22 0:00:00,897.0,6
4,776009710600603Mandla,77,6009710600603,64,77,2020-04-09 0:00:00,589.0,5
5,1156009684040641Margaret,115,6009684040641,97,115,2020-06-13 0:00:00,1716.0,4
6,1096001651262360Aqeela,109,6001651262360,92,109,2020-06-30 0:00:00,485.0,4
7,102300507946DIMPHO,102,300507946,87,102,2020-07-02 0:00:00,1523.0,4
8,996001106128616Melokuhle,99,6001106128616,84,99,2020-05-16 0:00:00,816.0,4
9,205010106111451Anna,20,5010106111451,18,20,2020-06-22 0:00:00,205.0,4


Q10) What is the total number of users that shop at Bhejane?

In [97]:
%%read_sql
SELECT COUNT(DISTINCT UserName)
FROM Users_3NF
LIMIT 50

Query started at 12:30:30 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT UserName)
0,158


Q11) What is the record count for the Colours_3NF Table?

In [101]:
%%read_sql
SELECT COUNT(DISTINCT colour)
FROM colours_3NF
LIMIT 50


Query started at 12:32:27 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT colour)
0,17


Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [104]:
%%read_sql
SELECT *
FROM Products_3NF
WHERE barcode = 'MPTAL57588104' OR barcode = '5000394203921' OR barcode = '6932391917652'
LIMIT 5


Query started at 12:34:27 AM E. Africa Standard Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Price,Quantity,Weight_kg,Volume_litre,Length,Width,Height,PathID,ItemID,ColourID,BrandID,LocationID
0,212,5000394203921,Duracell 2032 Speciality 3V Lithium Coin Batte...,35.0,316,,0.0,132.0,72.0,190.0,16,96,5,41.0,1.0
1,717,6932391917652,RGB Colourful Gaming Mouse Pad - Extra Large,329.0,30,,0.0,80.0,30.0,"0,3",61,345,5,137.0,NaN
2,718,6932391917652,RGB Colourful Gaming Mouse Pad - Extra Large,329.0,30,,0.0,80.0,30.0,"0,3",260,346,5,137.0,NaN
3,719,6932391917652,RGB Colourful Gaming Mouse Pad - Extra Large,329.0,30,,0.0,80.0,30.0,"0,3",60,347,5,137.0,NaN
4,1083,MPTAL57588104,5m Colour Changing RGB LED Strip Light,370.0,220,,0.0,5000.0,"0,5","0,01",281,516,16,NaN,NaN


Q13) What is the barcode of the most sold product?

In [115]:
%%read_sql
SELECT barcode, COUNT(barcode) AS counted
FROM Transactions_3NF
GROUP BY barcode
ORDER BY counted DESC
LIMIT 5;

Query started at 12:42:22 AM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,counted
0,4015400541813,4
1,MPTAL00404857,3
2,HURACANNANOWAVE1,3
3,6009686620537,3
4,6001865825405,3


Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [128]:
%%read_sql
SELECT C_3NF.CartID, U_3NF.username, C_3NF.invoicedate, P_3NF.productdescription
FROM Transactions_3NF AS T_3NF
LEFT JOIN Carts_3NF AS C_3NF
ON T_3NF.CartID = C_3NF.CartID
LEFT JOIN Users_3NF AS U_3NF
ON T_3NF.UserID = U_3NF.UserID
LEFT JOIN Products_3NF AS P_3NF
ON T_3NF.Barcode = P_3NF.Barcode
WHERE Username = 'Cornelis' AND InvoiceDate = '2020-06-28 0:00:00'

Query started at 12:57:12 AM E. Africa Standard Time; Query executed in 0.00 m

,CartID,UserName,InvoiceDate,ProductDescription
0,104,Cornelis,2020-06-28 0:00:00,Dettol Body Wash - Shower Gel - Original - 600ml
1,104,Cornelis,2020-06-28 0:00:00,Vitality Aloe Vera Gel 100ml
2,104,Cornelis,2020-06-28 0:00:00,Vitality Aloe Vera Gel 100ml
3,104,Cornelis,2020-06-28 0:00:00,Vitality Aloe Vera Gel 100ml
4,104,Cornelis,2020-06-28 0:00:00,Sparq Active Resistance Loop Band - Set of 5 w...
5,104,Cornelis,2020-06-28 0:00:00,Sparq Active Resistance Loop Band - Set of 5 w...
6,104,Cornelis,2020-06-28 0:00:00,Protective Face Shield
7,104,Cornelis,2020-06-28 0:00:00,Protective Face Shield


Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [163]:
%%read_sql
SELECT L_3NF.stockcountry, U_3NF.Username, T_3NF.cartid, T_3NF.barcode, C_3NF.invoicedate, SUBSTR(Invoicedate,6,2) AS month, 
    C_3NF.total, co_3NF.colour
FROM Transactions_3NF AS T_3NF
LEFT JOIN carts_3NF AS C_3NF
ON T_3NF.CartID = C_3NF.CartID
LEFT JOIN Products_3NF AS P_3NF
ON T_3NF.Barcode = P_3NF.Barcode
LEFT JOIN Colours_3NF AS co_3NF
ON P_3NF.ColourID = co_3NF.ColourID
LEFT JOIN Users_3NF AS U_3NF
ON T_3NF.UserID = U_3NF.UserID
LEFT JOIN Locations_3NF AS L_3NF
ON P_3NF.locationID = L_3NF.locationID
WHERE Colour = 'Black' AND month IN ('01', '03', '05', '07', '09', '11') AND stockcountry IS NOT NULL
LIMIT 50


Query started at 10:27:31 AM E. Africa Standard Time; Query executed in 0.00 m

,StockCountry,UserName,CartID,Barcode,InvoiceDate,month,Total,Colour
0,South Africa,Cindy,7,741311301174,2020-03-31 0:00:00,03,1999.0,Black
1,South Africa,Hester,18,5000394020146,2020-07-04 0:00:00,07,189.0,Black
2,South Africa,Conrad,32,6001246636682,2020-07-26 0:00:00,07,93.0,Black
3,South Africa,Carel,82,6925281939952,2020-05-04 0:00:00,05,799.0,Black
4,South Africa,Carel,82,6925281939952,2020-05-04 0:00:00,05,799.0,Black
5,South Africa,Carel,82,6925281939952,2020-05-04 0:00:00,05,799.0,Black
6,South Africa,Carel,82,6925281939952,2020-05-04 0:00:00,05,799.0,Black
7,South Africa,Carel,82,6925281939952,2020-05-04 0:00:00,05,799.0,Black
8,South Africa,Lacin,85,6941428152961,2020-05-18 0:00:00,05,798.0,Black
9,South Africa,Lacin,85,6941428152961,2020-05-18 0:00:00,05,798.0,Black
